In [ ]:
import requests
import pandas as pd
import math
import os
from scrapy import Selector
import csv

os.makedirs('book_data', exist_ok=True)
os.makedirs('book_images', exist_ok=True)

category_files = {}
category_writers = {}

i=1
while i < 3:
    url = "https://books.toscrape.com"
    next_page = "https://books.toscrape.com/catalogue/page-"+str(i)+".html"
    response = requests.get(next_page)
    sel = Selector(text=response.text)
    articles = sel.css("article.product_pod")

    print(next_page)

    for articles in articles :

        title = articles.css("a[href]::attr(title)").get()
        price = articles.css("div.product_price p.price_color::text").get()
        stock = articles.css("div.product_price p.availability::text")[1].get().strip()
        image = articles.css("div.image_container img::attr(src)").get()
        link = articles.css("div.image_container a::attr(href)").get()

        url_page = (url+"/catalogue/"+link)
        response = requests.get(url_page)
        sel_page = Selector(text=response.text)
        category = sel_page.css("ul.breadcrumb a[href]::text")[2].get()
        clean_category = category.replace(" ", "_").replace("/", "_").replace("\\", "_")

        image_url = url + "/" + image
        image_response = requests.get(image_url)

        category_image_dir = f"book_images/{clean_category}"
        os.makedirs(category_image_dir, exist_ok=True)

        if clean_category not in category_files:
            csv_file = open(f'book_data/{clean_category}.csv', 'w', newline='', encoding='utf-8')
            writer = csv.writer(csv_file)
            writer.writerow(['Title', 'Price', 'Stock', 'UPC', 'Category', 'Rating', 'Image_Path', 'Product_URL'])
            category_files[clean_category] = csv_file
            category_writers[clean_category] = writer

        rows = sel_page.css("tr")
        for row in rows[:1]:
            upc = row.css("td::text").getall()

        upc_str = upc[0] if upc else "unknown"
        image_filename = f"{upc_str}.jpg"
        image_path = os.path.join(category_image_dir, image_filename)


        with open(image_path, 'wb') as f:
            f.write(image_response.content)

        upc_str = upc[0] if upc else "unknown"
        image_filename = f"{upc_str}.jpg"
        image_path = os.path.join(category_image_dir, image_filename)

        rating_class = sel_page.css("p.star-rating::attr(class)").get()
        rating_text = "Unknown"
        rating_numeric = 0

        if 'One' in rating_class:
            rating_text = 'One Star / Five'
            rating_numeric = 1
        elif 'Two' in rating_class:
            rating_text = 'Two Stars / Five'
            rating_numeric = 2
        elif 'Three' in rating_class:
            rating_text = 'Three Stars / Five'
            rating_numeric = 3
        elif 'Four' in rating_class:
            rating_text = 'Four Stars / Five'
            rating_numeric = 4
        elif 'Five' in rating_class:
            rating_text = 'Five Stars / Five'
            rating_numeric = 5

        writer = category_writers[clean_category]
        writer.writerow([
            title,
            price,
            stock,
            upc[0] if upc else "",
            category,
            rating_numeric,
            image_path,
            url_page
        ])

        print(f"Title: {title}")
        print(f"Price: {price}")
        print(f"Stock: {stock}")
        print(f"Rating: {rating_text} ({rating_numeric}/5)")
        print(f"Category: {category}")
        print(f"Image saved: {image_path}")
        print(f"UPC: {upc[0] if upc else 'Unknown'}")
        print("-" * 50)
    i = i + 1

for file in category_files.values():
    file.close()

print("Scraping completed! Check the 'book_data' and 'book_images' folders.")

https://books.toscrape.com/catalogue/page-1.html
Title: A Light in the Attic
Price: Â£51.77
Stock: In stock
Rating: Three Stars / Five (3/5)
Category: Poetry
Image saved: book_images/Poetry\a897fe39b1053632.jpg
UPC: a897fe39b1053632
--------------------------------------------------
Title: Tipping the Velvet
Price: Â£53.74
Stock: In stock
Rating: One Star / Five (1/5)
Category: Historical Fiction
Image saved: book_images/Historical_Fiction\90fa61229261140a.jpg
UPC: 90fa61229261140a
--------------------------------------------------
Title: Soumission
Price: Â£50.10
Stock: In stock
Rating: One Star / Five (1/5)
Category: Fiction
Image saved: book_images/Fiction\6957f44c3847a760.jpg
UPC: 6957f44c3847a760
--------------------------------------------------
Title: Sharp Objects
Price: Â£47.82
Stock: In stock
Rating: Four Stars / Five (4/5)
Category: Mystery
Image saved: book_images/Mystery\e00eb4fd7b871a48.jpg
UPC: e00eb4fd7b871a48
--------------------------------------------------
Title: Sa